In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16, 'figure.figsize': (40, 8), 'font.family': 'serif', 'text.usetex': True, 'pgf.rcfonts': False})

%load_ext autoreload
%autoreload 2

from jax_smolyak.points import *
from jax_smolyak.indices import *
from jax_smolyak.smolyak_jax import MultivariateSmolyakBarycentricInterpolator

## Memory consumption of $F$ 

$F$ is the tensor that stores the target function evaluations and is dominating memory requirements.

### Independence of input and output dimension d (keeping the size of the multi-index set fixed at $n=5000$)

In [ ]:
def size_o(indxs, k, l, d) : # original, memory-efficient implementation
    size = 0
    for nu in indxs :
        zeta = smolyak_coefficient_zeta_sparse(lambda j : k[j], l, nu=nu, cutoff=d)
        if zeta != 0 :
            size += np.prod([si + 1 for si in nu.values()])
    return size

def size_p(indxs, k, l, d) : # using padding
    k_2_tau = {}
    k_2_n = {}
    for nu in indxs :
        zeta = smolyak_coefficient_zeta_sparse(lambda j : k[j], l, nu=nu, cutoff=d)
        if zeta != 0 :
            tau = tuple(nu.values())
            kk = len(tau)
            k_2_tau[kk] = tuple(max(nu1, nu2) for nu1, nu2 in zip(k_2_tau.get(kk, tau), tau))
            k_2_n[kk] = k_2_n.get(kk, 0) + 1

    size = 0
    for kk in k_2_tau.keys() :
        size += np.prod([si + 1 for si in k_2_tau[kk]]) * k_2_n[kk]
    return size

def size_pp(indxs, k, l, d) : # using both permutation and padding
    k_2_tau = {}
    k_2_n = {}
    for nu in indxs :
        zeta = smolyak_coefficient_zeta_sparse(lambda j : k[j], l, nu=nu, cutoff=d)
        if zeta != 0 :
            tau = tuple(sorted(nu.values(), reverse=True))
            kk = len(tau)
            k_2_tau[kk] = tuple(max(nu1, nu2) for nu1, nu2 in zip(k_2_tau.get(kk, tau), tau))
            k_2_n[kk] = k_2_n.get(kk, 0) + 1

    size = 0
    for kk in k_2_tau.keys() :
        size += np.prod([si + 1 for si in k_2_tau[kk]]) * k_2_n[kk]
    return size

In [ ]:
d_list = [10, 20, 40, 80, 160, 320, 640, 1250, 2500]
results = np.zeros((len(d_list), 3))

n = 5000
for i, d in enumerate(d_list) :
    k = [np.log((2+j)/np.log(2)) for j in range(d)]
    g = LejaMulti(d=d)
    l = find_suitable_l(k, n, nested=g.is_nested)
    indxs = indexset_sparse(lambda j : k[j], l, cutoff=d)
    results[i] = size_o(indxs, k, l, d), size_p(indxs, k, l, d), size_pp(indxs, k, l, d)

In [ ]:
fig, axs = plt.subplots(1,2)
axs[0].loglog(d_list, results[:, 0], marker='o', label='o')
axs[0].loglog(d_list, results[:, 1], marker='o', label='p')
axs[0].loglog(d_list, results[:, 2], marker='o', label='pp')
axs[1].loglog(d_list, results[:, 1]/results[:, 0], marker='o', label='p/o')
axs[1].loglog(d_list, results[:, 2]/results[:, 0], marker='o', label='pp/o')
for ax, title in zip(axs, ['absolute', 'relative', ]): ax.set_xlabel(r'$d$'); ax.set_ylabel(r'$d$'); ax.grid(); ax.set_title(title); ax.legend()
plt.tight_layout()

### In dependence of the size of the multi-index (keeping the dimension fixed at $d=200$)

In [ ]:
n_list = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600, 51200]
results = np.zeros((len(n_list), 3))

d = 200
for i, n in enumerate(n_list) :
    k = [np.log((2+j)/np.log(2)) for j in range(d)]
    g = LejaMulti(d=d)
    l = find_suitable_l(k, n, nested=g.is_nested)
    indxs = indexset_sparse(lambda j : k[j], l, cutoff=d)
    results[i] = size_o(indxs, k, l, d), size_p(indxs, k, l, d), size_pp(indxs, k, l, d)

In [ ]:
fig, axs = plt.subplots(1,2)
axs[0].loglog(n_list, results[:, 0], marker='o', label='o')
axs[0].loglog(n_list, results[:, 1], marker='o', label='p')
axs[0].loglog(n_list, results[:, 2], marker='o', label='pp')
axs[1].loglog(n_list, results[:, 1]/results[:, 0], marker='o', label='p/o')
axs[1].loglog(n_list, results[:, 2]/results[:, 0], marker='o', label='pp/o')
for ax, title in zip(axs, ['absolute', 'relative', ]): ax.set_xlabel(r'$n$'); ax.grid(); ax.set_title(title); ax.legend()
plt.tight_layout()